Оценка предобученных моделей распознавая текста на изображениях

Для изображений логотипов извлечение информации может быть полезным в случае наличия текстовых данных в логотипов.

Подобные алгоритмы могут быть полезны для оценки сходства входного логотоипа (или названия соответствующего бренда) с найденным в видеоролике при помощи известных метрик, например (расстояние Левенштейна)

In [1]:
import numpy as np
import pandas as pd 
import random
import torch

import matplotlib.pyplot as plt
from PIL import Image

import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

Seed

In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

Исследование предобученных моделей, которые можно использовать локально

Tesseract

In [3]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import pytesseract

Инференс

In [5]:
first_img_path = '/kaggle/input/example-images/adidas_shop.jpg'
second_img_path = '/kaggle/input/example-images/nike_logo.jpg'

print(pytesseract.image_to_string(Image.open(first_img_path)))

In [6]:
print(pytesseract.image_to_string(Image.open(second_img_path)))

 

 

JUST DOIT.



Здесь уже инференс сработал

Easy OCR

In [7]:
import easyocr
reader = easyocr.Reader(['en'])

In [8]:
results = reader.readtext(second_img_path)
for (bbox, text, prob) in results:
    print(f'Text: {text}, Probability: {prob}')


Text: JUST DOIi, Probability: 0.38044783663863674


In [9]:
results = reader.readtext(first_img_path)
for (bbox, text, prob) in results:
    print(f'Text: {text}, Probability: {prob}')

Text: clclces, Probability: 0.4152791785764064


Hugging Face API

Vision Encoder-Decoder model

In [10]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.47.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
image = Image.open(second_img_path).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

justport .


DocTR

In [ ]:
pip install inference-sdk

In [3]:
import os
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://infer.roboflow.com",
    api_key='gKkrveQc1gtDAoKcppQq'
)

result = CLIENT.ocr_image(inference_input=first_img_path)  
print(result)

Итак, для извлечения текстовой инфоромации об изображении имеет смысл использовать Roboflow DocTR API